# Forensica-AI: Deepfake Detection Training Notebook

This notebook contains all the code needed to train a deepfake detection model on Google Colab.

## Setup Instructions:
1. Upload this notebook to Google Colab
2. Paste your Kaggle dataset link in the cell below
3. Run all cells to start training

The model uses a CNN-RNN architecture to classify videos as real or fake.


In [12]:
# Install dependencies
%pip install -q kaggle opencv-python torch torchvision tqdm pyyaml scikit-learn pillow pandas numpy matplotlib


Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install facenet-pytorch


In [ ]:
from facenet_pytorch import MTCNN
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mtcnn = MTCNN(
    image_size=128,
    margin=20,
    keep_all=False,
    device=device
)


## Step 1: Download Dataset from Kaggle

Paste your Kaggle dataset link here. The dataset should contain videos in a structure like:
- `Celeb-real/` folder with real videos
- `Celeb-synthesis/` folder with fake videos


In [13]:
# ============================================
# KAGGLE DATASET DOWNLOAD
# ============================================
# Option 1: Using Kaggle API (recommended)
# First, upload your kaggle.json file or set credentials:
# from google.colab import files
# files.upload()  # Upload kaggle.json

# Option 2: Direct download link (paste your dataset link)
# Option 3: Manual upload via Colab file browser

import os
import zipfile
from pathlib import Path

# Configure your dataset here
KAGGLE_DATASET = "/kaggle/input/celeb-df-v2"  # Format: "username/dataset-name" (e.g., "tunguz/deepfake-detection")
DATASET_URL = ""  # Or paste direct download URL here

# Create directories
os.makedirs("data/raw_videos", exist_ok=True)
os.makedirs("data/frames", exist_ok=True)
os.makedirs("models", exist_ok=True)

print("📁 Directories created!")

# Download dataset if Kaggle dataset name is provided
if KAGGLE_DATASET:
    try:
        import kaggle
        print(f"📥 Downloading dataset: {KAGGLE_DATASET}")
        kaggle.api.dataset_download_files(KAGGLE_DATASET, path="data/", unzip=True)
        print("✅ Dataset downloaded!")
    except Exception as e:
        print(f"⚠️  Kaggle download failed: {e}")
        print("   Please upload your dataset manually to 'data/raw_videos' folder")
elif DATASET_URL:
    print(f"📥 Downloading from URL: {DATASET_URL}")
    import urllib.request
    urllib.request.urlretrieve(DATASET_URL, "dataset.zip")
    with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
        zip_ref.extractall("data/")
    print("✅ Dataset downloaded!")
else:
    print("\n⚠️  No dataset configured.")
    print("   Please either:")
    print("   1. Set KAGGLE_DATASET variable above (format: 'username/dataset-name')")
    print("   2. Set DATASET_URL variable above with direct download link")
    print("   3. Manually upload your dataset to the 'data/raw_videos' folder via Colab file browser")


📁 Directories created!
⚠️  Kaggle download failed: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
   Please upload your dataset manually to 'data/raw_videos' folder


## Step 2: Model Definitions

CNN Feature Extractor and RNN Classifier


In [14]:
# ============================================
# DATASET SETUP (Celeb-DF v2 - Attached Dataset)
# ============================================

import os
from pathlib import Path

# Root directory where the dataset is mounted
# (Kaggle/Colab-style input mount)
DATASET_ROOT = Path("/kaggle/input/celeb-df-v2")

# Dataset subfolders
CELEB_REAL_DIR = DATASET_ROOT / "Celeb-real"
CELEB_FAKE_DIR = DATASET_ROOT / "Celeb-synthesis"
YOUTUBE_REAL_DIR = DATASET_ROOT / "YouTube-real"

TEST_LIST_FILE = DATASET_ROOT / "List_of_testing_videos.txt"

# Sanity checks
assert CELEB_REAL_DIR.exists(), "Celeb-real folder not found"
assert CELEB_FAKE_DIR.exists(), "Celeb-synthesis folder not found"
assert YOUTUBE_REAL_DIR.exists(), "YouTube-real folder not found"
assert TEST_LIST_FILE.exists(), "List_of_testing_videos.txt not found"

print("✅ Celeb-DF v2 dataset found")
print(f"Real videos (Celeb): {len(list(CELEB_REAL_DIR.glob('*.mp4')))}")
print(f"Fake videos (Celeb): {len(list(CELEB_FAKE_DIR.glob('*.mp4')))}")
print(f"Real videos (YouTube): {len(list(YOUTUBE_REAL_DIR.glob('*.mp4')))}")

# Unified raw video directory (used by later cells)
RAW_VIDEO_DIR = DATASET_ROOT

✅ Celeb-DF v2 dataset found
Real videos (Celeb): 590
Fake videos (Celeb): 5639
Real videos (YouTube): 300


In [15]:
# ============================================
# CNN FEATURE EXTRACTOR (MobileNetV2 – FIXED)
# ============================================
import torch
import torch.nn as nn
from torchvision import models

class MobileNetFeatureExtractor(nn.Module):
    """
    MobileNetV2-based feature extractor
    Input:  [B*T, 3, H, W]
    Output: [B*T, 256]
    """
    def __init__(self, freeze_backbone=True):
        super().__init__()

        mobilenet = models.mobilenet_v2(pretrained=True)

        # Backbone
        self.backbone = mobilenet.features

        # Freeze early layers
        if freeze_backbone:
            for param in self.backbone.parameters():
                param.requires_grad = False

        # Projection head
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Sequential(
            nn.Linear(1280, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.4)
        )

        self.output_dim = 256

    def forward(self, x):
        x = self.backbone(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


def build_cnn(use_custom=False):
    """
    Build CNN feature extractor
    """
    model = MobileNetFeatureExtractor(freeze_backbone=True)
    return model, model.output_dim


print("✅ MobileNetV2 feature extractor ready!")


✅ CNN Feature Extractor defined!


In [16]:
# ============================================
# RNN CLASSIFIER
# ============================================
class RNNClassifier(nn.Module):
    """
    LSTM-based video classifier.
    Input:  [B, T, feature_dim]
    Output: [B, 2] (real/fake logits)
    """
    def __init__(self,
                 feature_dim=256,
                 hidden_size=128,
                 num_layers=1,
                 bidirectional=False,
                 dropout=0.3):
        super(RNNClassifier, self).__init__()
        self.feature_dim = feature_dim
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional

        self.lstm = nn.LSTM(
            input_size=feature_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=bidirectional,
            dropout=(dropout if num_layers > 1 else 0.0)
        )

        rnn_output_dim = hidden_size * (2 if bidirectional else 1)
        self.fc = nn.Linear(rnn_output_dim, 2)  # 2 classes: real/fake

    def forward(self, x):
        """
        x shape: [B, T, feature_dim]
        Returns: logits [B, 2]
        """
        out, (h_n, c_n) = self.lstm(x)
        
        if self.bidirectional:
            last_hidden = torch.cat((h_n[-2], h_n[-1]), dim=1)
        else:
            last_hidden = h_n[-1]

        logits = self.fc(last_hidden)
        return logits

print("✅ RNN Classifier defined!")


✅ RNN Classifier defined!


## Step 3: Dataset and Data Loading


In [ ]:
# ============================================
# VIDEO DATASET CLASS
# ============================================
import os
import random
from typing import List, Tuple
from torch.utils.data import Dataset, DataLoader, Subset, WeightedRandomSampler
from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np

NUM_FRAMES = 20
FRAME_SIZE = 128
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

LABEL_MAP = {
    "real": 0,
    "fake": 1,
}

frame_transform = transforms.Compose([
    transforms.Resize((FRAME_SIZE, FRAME_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])

class VideoSequenceDataset(Dataset):
    """
    PyTorch Dataset returning a fixed-length sequence of frames for each video.
    """
    def __init__(self, frames_root: str, labels_csv: str, num_frames: int = NUM_FRAMES,
                 transform=frame_transform, shuffle_frames: bool = False, cache_in_memory: bool = False):
        self.frames_root = frames_root
        self.labels_df = pd.read_csv(labels_csv)
        self.num_frames = num_frames
        self.transform = transform
        self.shuffle_frames = shuffle_frames
        self.cache_in_memory = cache_in_memory

        self.samples: List[Tuple[str, int]] = []
        for _, row in self.labels_df.iterrows():
            video_filename = row["video"]
            video_folder = os.path.splitext(video_filename)[0]
            folder_path = os.path.join(self.frames_root, video_folder)
            if not os.path.isdir(folder_path):
                continue
            available = len([f for f in os.listdir(folder_path) if f.lower().endswith((".jpg", ".png"))])
            if available < self.num_frames:
                continue
            label_str = str(row["label"]).lower()
            if label_str not in LABEL_MAP:
                continue
            label_int = LABEL_MAP[label_str]
            self.samples.append((video_folder, label_int))

        if len(self.samples) == 0:
            raise RuntimeError("No valid samples found. Check frames_root and labels_csv paths.")

        self._cache = {} if self.cache_in_memory else None
        if self.cache_in_memory:
            print("Caching frames in memory (may use lots of RAM)...")
            for vid, _ in self.samples:
                folder = os.path.join(self.frames_root, vid)
                frames = self._read_frames_from_folder(folder)
                self._cache[vid] = frames

    def _read_frames_from_folder(self, folder: str) -> List[Image.Image]:
        """Return list of PIL images sorted by frame index."""
        files = sorted([f for f in os.listdir(folder) if f.lower().endswith((".jpg", ".png"))])
        files = files[:self.num_frames]
        images = []
        for fname in files:
            path = os.path.join(folder, fname)
            img = Image.open(path).convert("RGB")
            images.append(img)
        return images

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        vid_folder, label = self.samples[idx]
        folder = os.path.join(self.frames_root, vid_folder)

        if self.cache_in_memory and vid_folder in self._cache:
            pil_frames = self._cache[vid_folder]
        else:
            pil_frames = self._read_frames_from_folder(folder)

        if self.shuffle_frames:
            pil_frames = pil_frames.copy()
            random.shuffle(pil_frames)

        frame_tensors = []
        for f in pil_frames:
            t = self.transform(f)
            frame_tensors.append(t)

        seq_tensor = torch.stack(frame_tensors, dim=0)
        label_tensor = torch.tensor(label, dtype=torch.long)

        return seq_tensor, label_tensor

def video_collate_fn(batch):
    """Collate function for video sequences."""
    seqs = [item[0] for item in batch]
    labels = torch.stack([item[1] for item in batch])
    batch_seqs = torch.stack(seqs, dim=0)
    return batch_seqs, labels

def build_loaders(frames_root: str, labels_csv: str, batch_size: int = 4, train_split: float = 0.8,
                  num_workers: int = 2, balanced_sampling: bool = True, **dataset_kwargs):
    dataset = VideoSequenceDataset(frames_root=frames_root, labels_csv=labels_csv, **dataset_kwargs)
    n = len(dataset)
    n_train = int(n * train_split)
    indices = list(range(n))
    random.shuffle(indices)
    train_idx = indices[:n_train]
    val_idx = indices[n_train:]

    train_set = Subset(dataset, train_idx)
    val_set = Subset(dataset, val_idx)

    if balanced_sampling:
        # Build a WeightedRandomSampler to balance real/fake classes in each batch
        train_labels = [dataset.samples[i][1] for i in train_idx]
        class_counts = np.bincount(train_labels, minlength=len(LABEL_MAP))
        # Avoid division by zero
        class_counts = np.where(class_counts == 0, 1, class_counts)
        class_weights = 1.0 / class_counts
        sample_weights = [class_weights[label] for label in train_labels]

        sampler = WeightedRandomSampler(
            weights=sample_weights,
            num_samples=len(sample_weights),
            replacement=True
        )

        train_loader = DataLoader(
            train_set,
            batch_size=batch_size,
            sampler=sampler,
            shuffle=False,
            num_workers=num_workers,
            collate_fn=video_collate_fn,
            pin_memory=True,
        )
    else:
        train_loader = DataLoader(
            train_set,
            batch_size=batch_size,
            shuffle=True,
            num_workers=num_workers,
            collate_fn=video_collate_fn,
            pin_memory=True,
        )

    val_loader = DataLoader(
        val_set,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        collate_fn=video_collate_fn,
        pin_memory=True,
    )

    return train_loader, val_loader

print("✅ Dataset class defined!")


✅ Dataset class defined!


## Step 4: Data Preparation

Prepare labels CSV and extract frames from videos


In [18]:
# ============================================
# PREPARE LABELS CSV (Celeb-DF v2 – Correct)
# ============================================
import csv
from pathlib import Path

def prepare_labels_csv(
    dataset_root="/kaggle/input/celeb-df-v2",
    labels_csv="data/labels.csv"
):
    """
    Prepare labels.csv for Celeb-DF v2
    
    REAL  = Celeb-real + YouTube-real
    FAKE  = Celeb-synthesis
    TEST  = excluded using List_of_testing_videos.txt
    """
    
    dataset_root = Path(dataset_root)
    labels_path = Path(labels_csv)
    labels_path.parent.mkdir(parents=True, exist_ok=True)

    celeb_real_dir = dataset_root / "Celeb-real"
    youtube_real_dir = dataset_root / "YouTube-real"
    celeb_fake_dir = dataset_root / "Celeb-synthesis"
    test_list_file = dataset_root / "List_of_testing_videos.txt"

    # Sanity checks
    assert celeb_real_dir.exists(), "Celeb-real folder missing"
    assert youtube_real_dir.exists(), "YouTube-real folder missing"
    assert celeb_fake_dir.exists(), "Celeb-synthesis folder missing"
    assert test_list_file.exists(), "List_of_testing_videos.txt missing"

    # Load official test videos
    with test_list_file.open("r") as f:
        test_videos = set(line.strip() for line in f if line.strip())

    samples = []

    # REAL videos (Celeb-real)
    for video_path in celeb_real_dir.glob("*.mp4"):
        if video_path.name not in test_videos:
            samples.append((video_path.name, "real"))

    # REAL videos (YouTube-real)
    for video_path in youtube_real_dir.glob("*.mp4"):
        if video_path.name not in test_videos:
            samples.append((video_path.name, "real"))

    # FAKE videos (Celeb-synthesis)
    for video_path in celeb_fake_dir.glob("*.mp4"):
        if video_path.name not in test_videos:
            samples.append((video_path.name, "fake"))

    if not samples:
        raise RuntimeError("No training videos found after filtering test set.")

    # Write CSV
    with labels_path.open("w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["video", "label"])
        writer.writerows(samples)

    print("✅ labels.csv created successfully")
    print(f"📊 Total videos: {len(samples)}")
    print(f"   Real: {sum(1 for _, l in samples if l == 'real')}")
    print(f"   Fake: {sum(1 for _, l in samples if l == 'fake')}")
    print(f"🚫 Test videos excluded: {len(test_videos)}")

    return labels_path


# Run preparation
labels_csv_path = prepare_labels_csv()
print(f"📄 Labels CSV saved at: {labels_csv_path}")

✅ labels.csv created successfully
📊 Total videos: 6529
   Real: 890
   Fake: 5639
🚫 Test videos excluded: 518
📄 Labels CSV saved at: data/labels.csv


In [19]:
# ============================================
# EXTRACT FRAMES FROM VIDEOS (Celeb-DF v2) – FIXED
# ============================================
import cv2
import os
from pathlib import Path
from tqdm import tqdm
import pandas as pd

# Load OpenCV face detector
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

def extract_frames_from_video(video_path, output_dir, num_frames=20, frame_size=128):
    if not video_path.exists():
        return False

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        return False

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames < num_frames + 10:
        cap.release()
        return False

    # Skip first 10% frames (often junk)
    start_frame = int(0.1 * total_frames)
    usable_frames = total_frames - start_frame
    interval = max(1, usable_frames // num_frames)

    output_dir.mkdir(parents=True, exist_ok=True)
    extracted = 0

    for i in range(num_frames):
        frame_idx = start_frame + i * interval
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if not ret:
            continue

        # Convert BGR → RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect face
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        if len(faces) > 0:
            x, y, w, h = faces[0]
            frame = frame[y:y+h, x:x+w]

        frame = cv2.resize(frame, (frame_size, frame_size))
        out_path = output_dir / f"frame_{i}.jpg"

        if cv2.imwrite(str(out_path), cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)):
            extracted += 1

    cap.release()
    return extracted == num_frames


def extract_all_frames(
    labels_csv="data/labels.csv",
    dataset_root="/kaggle/input/celeb-df-v2",
    frames_dir="data/frames",
    num_frames=20,
    frame_size=128
):
    labels = pd.read_csv(labels_csv)
    dataset_root = Path(dataset_root)
    frames_dir = Path(frames_dir)
    frames_dir.mkdir(parents=True, exist_ok=True)

    success_count, skip_count = 0, 0

    for _, row in tqdm(labels.iterrows(), total=len(labels), desc="Extracting"):
        video_name = row["video"]

        candidate_paths = [
            dataset_root / "Celeb-real" / video_name,
            dataset_root / "YouTube-real" / video_name,
            dataset_root / "Celeb-synthesis" / video_name,
        ]

        video_path = next((p for p in candidate_paths if p.exists()), None)
        if video_path is None:
            skip_count += 1
            continue

        output_dir = frames_dir / video_path.stem
        ok = extract_frames_from_video(
            video_path,
            output_dir,
            num_frames=num_frames,
            frame_size=frame_size
        )

        if ok:
            success_count += 1
        else:
            skip_count += 1

    print("\n✅ Frame extraction completed")
    print(f"   Success: {success_count}")
    print(f"   Skipped: {skip_count}")

    return success_count, skip_count


# RUN
success, skipped = extract_all_frames()


🎬 Extracting frames from 6529 videos
   Frames/video: 20, Size: 128x128


Extracting frames: 100%|██████████| 6529/6529 [19:23<00:00,  5.61it/s]


✅ Frame extraction finished
   Successfully processed: 6528
   Skipped: 1


## Step 5: Training Configuration


In [20]:
# ============================================
# TRAINING CONFIGURATION
# ============================================
# You can modify these hyperparameters

CONFIG = {
    "batch_size": 4,
    "num_frames": 20,
    "frame_size": 128,
    "learning_rate": 0.0003,
    "epochs": 15,
    "lstm_hidden_size": 128,
    "num_layers": 1,
    "train_split": 0.8,
    "frames_dir": "data/frames",
    "labels_csv": "data/labels.csv",
}

print("📋 Training Configuration:")
for key, value in CONFIG.items():
    print(f"   {key}: {value}")


📋 Training Configuration:
   batch_size: 4
   num_frames: 20
   frame_size: 128
   learning_rate: 0.0003
   epochs: 15
   lstm_hidden_size: 128
   num_layers: 1
   train_split: 0.8
   frames_dir: data/frames
   labels_csv: data/labels.csv


## Step 6: Training Functions


In [21]:
# ============================================
# TRAINING FUNCTIONS
# ============================================
import torch.optim as optim
from tqdm import tqdm

def train_one_epoch(cnn, rnn, loader, criterion, optimizer, device):
    cnn.train()
    rnn.train()

    total_loss = 0
    correct = 0
    total = 0

    for seqs, labels in tqdm(loader, desc="Training", ncols=100):
        seqs = seqs.to(device)
        labels = labels.to(device)

        B, T, C, H, W = seqs.shape
        seqs_reshaped = seqs.view(B * T, C, H, W)
        features = cnn(seqs_reshaped)
        feature_dim = features.shape[-1]
        features = features.view(B, T, feature_dim)

        logits = rnn(features)
        loss = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * B
        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total += B

    return total_loss / total, correct / total

def validate(cnn, rnn, loader, criterion, device):
    cnn.eval()
    rnn.eval()

    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for seqs, labels in tqdm(loader, desc="Validating", ncols=100):
            seqs = seqs.to(device)
            labels = labels.to(device)

            B, T, C, H, W = seqs.shape
            seqs_reshaped = seqs.view(B * T, C, H, W)
            features = cnn(seqs_reshaped)
            feature_dim = features.shape[-1]
            features = features.view(B, T, feature_dim)

            logits = rnn(features)
            loss = criterion(logits, labels)

            total_loss += loss.item() * B
            preds = torch.argmax(logits, dim=1)
            correct += (preds == labels).sum().item()
            total += B

    return total_loss / total, correct / total

print("✅ Training functions defined!")


✅ Training functions defined!


## Step 7: Start Training

Run this cell to begin training your model!


In [ ]:
# ============================================
# MAIN TRAINING LOOP
# ============================================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️  Using device: {device}")

# Build data loaders
print("\n📦 Building data loaders...")
train_loader, val_loader = build_loaders(
    frames_root=CONFIG["frames_dir"],
    labels_csv=CONFIG["labels_csv"],
    batch_size=CONFIG["batch_size"],
    train_split=CONFIG["train_split"],
    num_workers=2,
    num_frames=CONFIG["num_frames"],
    cache_in_memory=False,
    balanced_sampling=True,
)

print(f"   Train samples: {len(train_loader.dataset)}")
print(f"   Val samples: {len(val_loader.dataset)}")

# Build models
print("\n🏗️  Building models...")
cnn, feature_dim = build_cnn()

cnn.to(device)

rnn = RNNClassifier(
    feature_dim=feature_dim,
    hidden_size=CONFIG["lstm_hidden_size"],
    num_layers=CONFIG["num_layers"],
    bidirectional=False
)
rnn.to(device)

print(f"   CNN feature dim: {feature_dim}")
print(f"   RNN hidden size: {CONFIG['lstm_hidden_size']}")

# Loss and optimizer
# Stronger weighting for the minority REAL class to avoid always predicting FAKE
class_weights = torch.tensor([3.0, 1.0]).float().to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

optimizer = optim.Adam(
    list(cnn.parameters()) + list(rnn.parameters()),
    lr=CONFIG["learning_rate"]
)

# Training loop
print(f"\n🚀 Starting training for {CONFIG['epochs']} epochs...")
print("="*60)

best_val_acc = 0.0
SAVE_DIR = "models"
os.makedirs(SAVE_DIR, exist_ok=True)

for epoch in range(CONFIG["epochs"]):
    print(f"\n--- Epoch {epoch + 1}/{CONFIG['epochs']} ---")

    train_loss, train_acc = train_one_epoch(
        cnn, rnn, train_loader, criterion, optimizer, device
    )
    val_loss, val_acc = validate(
        cnn, rnn, val_loader, criterion, device
    )

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.4f}")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_path = os.path.join(SAVE_DIR, "best_model.pth")
        best_model_data = {
            "cnn_state": cnn.state_dict(),
            "rnn_state": rnn.state_dict(),
            "feature_dim": feature_dim,
            "rnn_params": {
                "feature_dim": feature_dim,
                "hidden_size": CONFIG["lstm_hidden_size"],
                "num_layers": CONFIG["num_layers"],
                "bidirectional": False
            },
            "config": CONFIG,
            "best_val_acc": best_val_acc,
            "epoch": epoch + 1,
            "val_loss": val_loss,
            "val_acc": val_acc
        }
        torch.save(best_model_data, best_model_path)
        print(f"✅ Best model saved (acc={val_acc:.4f}) at epoch {epoch + 1}")

print("\n" + "="*60)
print("🎉 Training complete!")
print(f"Best validation accuracy: {best_val_acc:.4f}")
print(f"Model saved to: {os.path.join(SAVE_DIR, 'best_model.pth')}")
print("="*60)


🖥️  Using device: cuda

📦 Building data loaders...
   Train samples: 5222
   Val samples: 1306

🏗️  Building models...
   CNN feature dim: 256
   RNN hidden size: 128

🚀 Starting training for 15 epochs...

--- Epoch 1/15 ---


Validating: 100%|█████████████████████████████████████████████████| 327/327 [01:06<00:00,  4.94it/s]


Train Loss: 0.4079 | Train Acc: 0.8602
Val   Loss: 0.3715 | Val   Acc: 0.8783
✅ Best model saved (acc=0.8783) at epoch 1

--- Epoch 2/15 ---


Validating: 100%|█████████████████████████████████████████████████| 327/327 [01:06<00:00,  4.94it/s]


Train Loss: 0.4046 | Train Acc: 0.8602
Val   Loss: 0.3817 | Val   Acc: 0.8783

--- Epoch 3/15 ---


...

Train Loss: 0.4058 | Train Acc: 0.8602
Val   Loss: 0.3725 | Val   Acc: 0.8783

--- Epoch 14/15 ---


Validating: 100%|█████████████████████████████████████████████████| 327/327 [01:05<00:00,  4.97it/s]


Train Loss: 0.4055 | Train Acc: 0.8602
Val   Loss: 0.3717 | Val   Acc: 0.8783

--- Epoch 15/15 ---


Validating: 100%|█████████████████████████████████████████████████| 327/327 [01:06<00:00,  4.94it/s]

Train Loss: 0.4054 | Train Acc: 0.8602
Val   Loss: 0.3709 | Val   Acc: 0.8783

🎉 Training complete!
Best validation accuracy: 0.8783
Model saved to: models/best_model.pth


In [24]:
# ============================================
# EVALUATION
# ============================================
from sklearn.metrics import confusion_matrix, classification_report

MODEL_PATH = "models/best_model.pth"

if os.path.exists(MODEL_PATH):
    print("📊 Evaluating model...")
    
    checkpoint = torch.load(MODEL_PATH, map_location=device)
    feature_dim = checkpoint["feature_dim"]
    
    cnn, _ = build_cnn(use_custom=True)
    rnn = RNNClassifier(feature_dim=feature_dim)
    
    cnn.load_state_dict(checkpoint["cnn_state"])
    rnn.load_state_dict(checkpoint["rnn_state"])
    
    cnn.to(device)
    rnn.to(device)
    
    cnn.eval()
    rnn.eval()
    
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for seqs, labels in tqdm(val_loader, desc="Evaluating", ncols=100):
            seqs = seqs.to(device)
            labels = labels.to(device)
            
            B, T, C, H, W = seqs.shape
            seqs_reshaped = seqs.view(B*T, C, H, W)
            
            features = cnn(seqs_reshaped)
            features = features.view(B, T, -1)
            
            logits = rnn(features)
            preds = torch.argmax(logits, dim=1)
            
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    
    print("\n📈 Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=["real", "fake"]))
    
    print("\n📊 Confusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))
    
    acc = np.mean(np.array(all_labels) == np.array(all_preds))
    print(f"\n✅ Final Validation Accuracy: {acc:.4f}")
else:
    print("❌ Model not found. Please train the model first.")


📊 Evaluating model...


Evaluating: 100%|█████████████████████████████████████████████████| 327/327 [01:07<00:00,  4.86it/s]


📈 Classification Report:
              precision    recall  f1-score   support

        real       0.00      0.00      0.00       159
        fake       0.88      1.00      0.94      1147

    accuracy                           0.88      1306
   macro avg       0.44      0.50      0.47      1306
weighted avg       0.77      0.88      0.82      1306


📊 Confusion Matrix:
[[   0  159]
 [   0 1147]]

✅ Final Validation Accuracy: 0.8783



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Step 9: Inference on New Videos

Use this cell to predict on a single video


In [ ]:
# ============================================
# INFERENCE ON SINGLE VIDEO
# ============================================
from torchvision import transforms
from PIL import Image

def extract_frames_inference(video_path, num_frames=20):
    """Extract frames from a single video for inference."""
    cap = cv2.VideoCapture(video_path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    if total < num_frames:
        raise ValueError("Video too short for inference!")
    
    interval = total // num_frames
    frames = []
    
    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * interval)
        ret, frame = cap.read()
        if not ret:
            continue
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = Image.fromarray(frame)
        frames.append(frame)
    
    cap.release()
    return frames

def predict_video(video_path, model_path="models/best_model.pth"):
    """Predict if a video is real or fake."""
    if not os.path.exists(model_path):
        print("❌ Model not found. Please train the model first.")
        return None
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Load model
    checkpoint = torch.load(model_path, map_location=device)
    feature_dim = checkpoint["feature_dim"]
    
    cnn, _ = build_cnn()

    rnn = RNNClassifier(feature_dim=feature_dim)
    
    cnn.load_state_dict(checkpoint["cnn_state"])
    rnn.load_state_dict(checkpoint["rnn_state"])
    
    cnn.to(device)
    rnn.to(device)
    
    cnn.eval()
    rnn.eval()
    
    # Extract frames
    frames = extract_frames_inference(video_path)
    
    # Preprocess
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    tensors = []
    for f in frames:
        tensors.append(transform(f))
    
    seq_tensor = torch.stack(tensors, dim=0)  # [T, C, H, W]
    seq_tensor = seq_tensor.unsqueeze(0)      # [1, T, C, H, W]
    
    # Predict
    with torch.no_grad():
        B, T, C, H, W = seq_tensor.shape
        seq_tensor = seq_tensor.to(device)
        
        reshaped = seq_tensor.view(B*T, C, H, W)
        features = cnn(reshaped)
        features = features.view(B, T, -1)
        
        logits = rnn(features)
        probs = torch.softmax(logits, dim=1)
    
    pred_class = torch.argmax(probs, dim=1).item()
    confidence = probs[0][pred_class].item()
    
    label_map = {0: "REAL", 1: "FAKE"}
    
    print("\n" + "="*40)
    print("🎬 Video Prediction")
    print("="*40)
    print(f"Video: {video_path}")
    print(f"Prediction: {label_map[pred_class]}")
    print(f"Confidence: {confidence:.4f}")
    print("="*40)
    
    return label_map[pred_class], confidence

# Example usage (uncomment and provide video path):
 #video_path = "/kaggle/input/celeb-df-v2/Celeb-synthesis/id0_id16_0002.mp4"
predict_video('/kaggle/input/celeb-df-v2/Celeb-real/id0_0008.mp4')

print("✅ Inference function ready!")
print("   Use: predict_video('/kaggle/input/celeb-df-v2/Celeb-synthesis/id0_id16_0002.mp4')")



🎬 Video Prediction
Video: /kaggle/input/celeb-df-v2/Celeb-real/id0_0008.mp4
Prediction: FAKE
Confidence: 0.8624
✅ Inference function ready!
   Use: predict_video('/kaggle/input/celeb-df-v2/Celeb-synthesis/id0_id16_0002.mp4')


## Download Trained Model

Download your trained model from Colab


In [27]:
# Download the trained model
from google.colab import files

if os.path.exists("models/best_model.pth"):
    print("📥 Downloading best_model.pth...")
    files.download("models/best_model.pth")
    print("✅ Download complete!")
else:
    print("❌ Model not found. Please train the model first.")


📥 Downloading best_model.pth...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download complete!
